# Library

In [1]:
# 데이터 처리 및 분석을 위한 라이브러리
import numpy as np  # 수학 연산 및 배열 연산을 위한 라이브러리
import pandas as pd  # 데이터 프레임을 다루기 위한 라이브러리

# 데이터 시각화를 위한 라이브러리
import matplotlib.pyplot as plt  # 그래프 및 차트 그리기
import seaborn as sns  # 시각화 기능을 향상시키는 라이브러리

# 머신러닝 관련 라이브러리
from sklearn.cluster import KMeans  # K-means 클러스터링 알고리즘 (비지도 학습)
from sklearn.preprocessing import LabelEncoder, StandardScaler  # 데이터 전처리를 위한 도구
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF 벡터 변환 (텍스트 데이터 벡터화)
from sklearn.metrics.pairwise import sigmoid_kernel  # 시그모이드 커널을 이용한 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도를 계산하는 함수

# 추천 시스템 관련 라이브러리 (Surprise 라이브러리 사용)
from surprise import SVD  # SVD(특이값 분해) 기반 추천 시스템 알고리즘
from surprise import Dataset, Reader  # 데이터셋 로딩 및 처리
from surprise.model_selection import train_test_split  # 추천 시스템용 데이터 분할
from surprise import accuracy  # 추천 시스템 평가 (RMSE 등 측정)

# 경고 메시지 무시 (불필요한 경고를 숨기기 위해 사용)
import warnings
warnings.filterwarnings('ignore')

# 자연어 처리 관련 라이브러리
import nltk  # 자연어 처리(NLP)를 위한 라이브러리
import re  # 정규 표현식 (문자열 처리)
import string  # 문자열 관련 기능 제공
from nltk.tokenize import word_tokenize  # 문장을 단어 단위로 토큰화
from nltk.corpus import stopwords  # 불용어(의미 없는 단어) 제거
from nltk.stem import PorterStemmer  # 어간 추출 (동사의 변형을 정규화)

# 실행 시간 측정 (성능 비교 등 활용)
import time

# 최근접 이웃 알고리즘을 위한 라이브러리
from scipy.sparse import csr_matrix  # 희소 행렬(대부분이 0인 행렬) 변환
from sklearn.neighbors import NearestNeighbors  # 최근접 이웃 알고리즘 (KNN 등)


---

# Load Data

In [2]:
rating = pd.read_csv('./data/rating.csv')
anime = pd.read_csv('./data/anime.csv')

---

# Data Summary

In [3]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [4]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
anime[anime.name=='Death Note']
# print(anime[anime.name=='One Punch Man'])
# print(anime[anime.name=='One Piece'])

,anime_id,name,genre,type,episodes,rating,members
40,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917


In [6]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [8]:
print(f'anime shape: {anime.shape}\nrating shape: {rating.shape}')

anime shape: (12294, 7)
rating shape: (7813737, 3)


---

# Check Missing Values

In [9]:
rating.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [10]:
anime.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

---

# Remove Missing Rows

In [ ]:
anime.dropna(axis=0, inplace=True)
anime.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [12]:
anime.describe()

,anime_id,rating,members
count,12017.000000,12017.000000,1.201700e+04
mean,13638.001165,6.478264,1.834888e+04
std,11231.076675,1.023857,5.537250e+04
min,1.000000,1.670000,1.200000e+01
25%,3391.000000,5.890000,2.250000e+02
50%,9959.000000,6.570000,1.552000e+03
75%,23729.000000,7.180000,9.588000e+03
max,34519.000000,10.000000,1.013917e+06


In [13]:
anime.episodes.value_counts()

episodes
1      5571
2      1075
12      810
13      571
26      514
       ... 
358       1
366       1
201       1
172       1
125       1
Name: count, Length: 187, dtype: int64

---

# Check Duplicates

In [14]:
duplicated_anime = anime[anime.duplicated()].shape[0] #.shape[0] → 데이터프레임의 행(row) 개수를 의미
#anime[anime.duplicated()] -> duplicated()가 True인 행만 선택해서 새로운 데이터프레임을 만듦.
#anime.duplicated() -> 중복된 행인지(True/False) 확인
print(f'count of duplicate anime: {duplicated_anime}')

count of duplicate anime: 0


In [15]:
duplicated_rating = rating[rating.duplicated()].shape[0]
print(f'count of dupliacte anime: {duplicated_rating}') # 찐빠 발생

count of dupliacte anime: 1


---

# Remove Duplicates

In [16]:
rating.drop_duplicates(keep='first', inplace=True) # 첫 번쨰 등장한 값 유지

duplicated_rating = rating[rating.duplicated()].shape[0]
print(f'count of duplicated anime after removing: {duplicated_rating}')

count of duplicated anime after removing: 0


---

# Create Dateset

In [17]:
df = pd.merge(anime, rating, on='anime_id')
df.to_csv("./data/anime_rating_merged.csv", index=False)

In [18]:
# df = pd.read_csv('./data/anime_rating_merged.csv')
# df27364 = df[df['user_id'] == 27364]
# df27364.to_csv("./data/anime_rating_27364.csv", index=False)

In [19]:
# df27364.tail()

In [20]:
df.tail(20)

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
7813590,5541,The Satisfaction,Hentai,OVA,1,4.37,166,39532,-1
7813591,5541,The Satisfaction,Hentai,OVA,1,4.37,166,48766,-1
7813592,5541,The Satisfaction,Hentai,OVA,1,4.37,166,58483,1
7813593,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,20171,7
7813594,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,39532,-1
7813595,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,48766,-1
7813596,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,58483,1
7813597,5543,Under World,Hentai,OVA,1,4.28,183,39532,-1
7813598,5543,Under World,Hentai,OVA,1,4.28,183,48766,-1
7813599,5543,Under World,Hentai,OVA,1,4.28,183,49503,4


In [21]:
# df = df.rename(columns={'rating_x': 'user_rating'})
# df = df.drop('rating_y', axis=1)

# df.to_csv('anime_rating.csv', index=False) # 인덱스 미포함.

In [22]:
df = df.rename(columns={'rating_x': 'anime_rating'})
df = df.rename(columns={'rating_y': 'user_rating'})
df = df[df['user_rating'] != -1] # user_rating이 -1인 행 제거
df.to_csv("./data/anime_rating_-1.csv", index=False)
print("user_rating=-1 개수:", (df['user_rating'] == -1).sum())

user_rating=-1 개수: 0


In [23]:
df = pd.read_csv('./data/anime_rating_-1.csv')
print(f'dataset shape: {df.shape}')

dataset shape: (6337145, 9)


In [24]:
df.head(20)

,anime_id,name,genre,type,episodes,anime_rating,members,user_id,user_rating
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,322,10
5,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,398,10
6,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,462,8
7,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,490,10
8,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,548,10
9,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,570,10


In [25]:
df.describe()

,anime_id,anime_rating,members,user_id,user_rating
count,6.337145e+06,6.337145e+06,6.337145e+06,6.337145e+06,6.337145e+06
mean,8.902547e+03,7.675013e+00,1.845767e+05,3.674795e+04,7.808543e+00
std,8.881674e+03,6.699057e-01,1.909532e+05,2.101337e+04,1.572440e+00
min,1.000000e+00,2.000000e+00,3.300000e+01,1.000000e+00,1.000000e+00
25%,1.239000e+03,7.290000e+00,4.680300e+04,1.898500e+04,7.000000e+00
50%,6.213000e+03,7.700000e+00,1.170910e+05,3.681500e+04,8.000000e+00
75%,1.407500e+04,8.150000e+00,2.563250e+05,5.487300e+04,9.000000e+00
max,3.447500e+04,9.370000e+00,1.013917e+06,7.351600e+04,1.000000e+01


In [26]:
# df = pd.read_csv("./data/anime_rating.csv")

# # user_id가 27364인 데이터만 필터링
# df_filtered = df[df['user_id'] == 27364]

# df_filtered.to_csv("./data/anime_rating_27364.csv", index=False)

---

# Preprocessing Function

In [27]:
df = df.copy() #데이터프레임을 복사하여 원본을 보호, 원본 데이터프레임(df)을 직접 변경하는 것이 아니라 안전하게 수정 가능.
df['user_rating'].replace(to_replace=-1, value=np.nan, inplace=True) #user_rating 컬럼에서 -1 값을 NaN(결측값)으로 변환
df = df.dropna(axis=0) # NaN 이 포함된 행(row) 삭제
print("Null values after final pre-processing:")
df.isna().sum() # 각 컬럼별로 결측값 개수를 출력

Null values after final pre-processing:


anime_id        0
name            0
genre           0
type            0
episodes        0
anime_rating    0
members         0
user_id         0
user_rating     0
dtype: int64

In [28]:
def lower_text(text): # lower_text 함수선언, text라는 입력값(문자열)을 받음.
    """
        to lowercase # 함수 설명 작성 -> 소문자로 변환 가능
    """
    text = text.lower() # 모든 문자 소문자로 변환
    return text #소문자로 변환된 문자열 반환 

# 왜 소문자로 변환할까?
# Naruto와 naruto를 같은 단어로 인식하기 위해!
# 머신러닝/딥러닝 모델이 불필요한 차이를 학습하지 않도록!

In [29]:
def clean_text(text):
    """
        data preprocessing 
    """
    
    # to lowercase
    text = text.lower()

    # remove sybmols and other words
    text = re.sub(r'<[^>]*>', '', text) # <html> 같은 태그 제거
    text = re.sub(r'http\S+', '', text) # URL 제거
    text = re.sub(r'&quot;', '', text) # 특수 기호 제거
    text = re.sub(r'.hack//', '', text) # ".hack//"같은 패턴 제거
    text = re.sub(r'&#039;', '', text) # '&#039;' -> '' (어포스트로피 깨짐 현상 제거)
    text = re.sub(r'A&#039;s', '', text) # A&#039;s -> ''
    text = re.sub(r'I&#039;', 'I\'', text) # 'I&#039;' → 'I\'' (아포스트로피 복구)
    text = re.sub(r'&amp;', 'and', text) # '&amp;' → 'and' (HTML 인코딩 복구)
  
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 4. 숫자 제거 (현재는 주석 처리됨, 필요시 활성화)
    # text = re.sub(r'\d+', '', text)

    # 5. 토큰화 (단어 단위로 분리)
    # words = word_tokenize(text)

    # 6. 불용어 제거 (stopwords)
    # stop_words = set(stopwords.words('english'))
    # words = [word for word in words if word not in stop_words]

    # 7. 어간 추출 (stemming)
    # stemmer = PorterStemmer()
    # words = [stemmer.stem(word) for word in words]

    # 8. 다시 하나의 문자열로 합치기
    # text = ' '.join(words)
    
    return text

---

# Data Preprocessing

In [30]:
# start_time = time.time()
# df['name']=df['name'].apply(clean_text)
# anime['name']=anime['name'].apply(clean_text)
# end_time = time.time()
# elapsed_time = end_time - start_time
# print("process time:", elapsed_time, "sec.")

start_time = time.time() #time.time()을 사용하여 코드 실행이 시작되는 시간을 저장
df['name']=df['name'].apply(clean_text) #df의 "name" 컬럼의 모든 값에 clean_text() 적용
anime['name'] = anime['name'].apply(clean_text) #anime 데이터프레임에도 동일한 작업 수행
end_time = time.time() #실행이 끝나는 시점의 시간 기록
elapsed_time = end_time - start_time #실행 시간을 초 단위로 계산 
print("process time: ", elapsed_time, " sec.")

process time:  37.543956995010376  sec.


---

# Popularity-Based Recommender 인기 기반 추천

### User rating 순위 뽑아보기

In [31]:
def popularity_recommender_u(df, selected_features):
    """
        recommender system with popularity-based
    """
    # grouping & calculating mean value 
    grouped_df = df.groupby(selected_features).agg({'user_rating': 'mean'}).reset_index()
    # sorting to rating
    sorted_df = grouped_df.sort_values('user_rating', ascending=False)
    # give the recommedations
    recommendations = sorted_df.head(15)
    return recommendations

In [32]:
df.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'anime_rating',
       'members', 'user_id', 'user_rating'],
      dtype='object')

In [33]:
df.head(10)

,anime_id,name,genre,type,episodes,anime_rating,members,user_id,user_rating
0,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5
1,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10
2,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10
3,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10
4,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,322,10
5,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,398,10
6,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,462,8
7,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,490,10
8,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,548,10
9,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,570,10


In [34]:
# according to anime names
selected_features = ['name']
popularity_recommender_u(df, selected_features)

,name,user_rating
1644,dededen,10.0
8454,takoyaki mantman,10.0
3040,hamster club,10.0
8228,star beat hoshi no kodou,10.0
3201,hello kitty no circus ga yatte kita,10.0
3209,hello kitty no mahou no ringo,10.0
3219,hello kitty no tomatta big ben,10.0
3220,hello kitty no yappari mama ga suki,10.0
9591,yattokame tanteidan,10.0
7921,shiroi zou,10.0


In [35]:
# according to members
selected_features = ['members']
popularity_recommender_u(df, selected_features)

,members,user_rating
3391,8028,9.500000
6004,114262,9.449495
6256,200630,9.426313
5749,80679,9.389788
6467,793665,9.322741
6140,151266,9.272552
6463,673572,9.261326
6396,336376,9.236398
6428,425855,9.234586
5757,81109,9.202258


----

### Anime rating 뽑아보기

In [36]:
def popularity_recommender_a(df, selected_features):
    """
        recommender system with popularity-based
    """
    # grouping & calculating mean value 
    grouped_df = df.groupby(selected_features).agg({'anime_rating': 'mean'}).reset_index()
    # sorting to rating
    sorted_df = grouped_df.sort_values('anime_rating', ascending=False)
    # give the recommedations
    recommendations = sorted_df.head(15)
    return recommendations

In [37]:
selected_features = ['name']
popularity_recommender_a(df, selected_features)

,name,anime_rating
4383,kimi no na wa,9.370000
2357,fullmetal alchemist brotherhood,9.260000
2740,gintama°,9.250000
8245,steinsgate,9.170000
2976,haikyuu karasuno koukou vs shiratorizawa gakue...,9.150000
3523,hunter x hunter 2011,9.130000
2730,gintama enchousen,9.110000
2687,ginga eiyuu densetsu,9.110000
2733,gintama movie kanketsuhen yorozuya yo eien nare,9.100000
2728,gintama,9.090657


In [38]:
# according to members
selected_features = ['members']
popularity_recommender_a(df, selected_features)

,members,anime_rating
6256,200630,9.37
6467,793665,9.26
6004,114262,9.25
6463,673572,9.17
6140,151266,9.16
5868,93351,9.15
6428,425855,9.13
5749,80679,9.11
5757,81109,9.11
5669,72534,9.10


---

### First genre 생성

In [39]:
# create first genre
df['first_genre'] = df['genre'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x)

In [40]:
df.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'anime_rating',
       'members', 'user_id', 'user_rating', 'first_genre'],
      dtype='object')

In [41]:
selected_features = ['first_genre']
popularity_recommender_u(df, selected_features)

,first_genre,user_rating
14,Josei,8.574034
28,Sci-Fi,8.502633
21,Mystery,8.355527
24,Psychological,8.327117
6,Drama,7.952341
9,Game,7.870670
0,Action,7.867908
4,Dementia,7.863052
1,Adventure,7.798738
2,Cars,7.756006


In [42]:
selected_features = ['first_genre']
popularity_recommender_a(df, selected_features)

,first_genre,anime_rating
14,Josei,8.469407
28,Sci-Fi,8.389518
24,Psychological,8.219053
21,Mystery,8.214965
4,Dementia,7.848917
6,Drama,7.810982
9,Game,7.764884
0,Action,7.733717
1,Adventure,7.703489
2,Cars,7.689872


In [43]:
#according to type
selected_features = ['type']
popularity_recommender_u(df, selected_features)

,type,user_rating
0,Movie,7.922580
5,TV,7.899160
4,Special,7.463638
3,OVA,7.334584
2,ONA,7.229329
1,Music,7.214282


In [44]:
#according to type
selected_features = ['type']
popularity_recommender_a(df, selected_features)

,type,anime_rating
0,Movie,7.832864
5,TV,7.753773
4,Special,7.349545
3,OVA,7.215986
2,ONA,7.069045
1,Music,7.049417


# Search Function

In [45]:
def search_anime(anime_name):
    """
    애니 이름에 특정 단어가 포함된 모든 애니 검색
    """
    filtered_anime = df[df["name"].str.contains(anime_name, case=False, na=False)]
    
    if filtered_anime.empty:
        return f"'{anime_name}'을 포함하는 애니를 찾을 수 없습니다."
    
    # "anime_rating" 중복 제거 & "user_rating" 평균 처리
    filtered_anime = filtered_anime.groupby("name", as_index=False).agg({
        "genre": "first",  # 첫 번째 값 사용
        "type": "first",  # 첫 번째 값 사용
        "episodes": "first",  # 첫 번째 값 사용
        "anime_rating": "first",  # 중복 제거 (첫 번째 값 선택)
        "user_rating": "mean",  # 중복된 경우 평균 계산
        "members": "first",  # 첫 번째 값 사용
    })

    return filtered_anime[["name", "genre", "type", "episodes", "anime_rating", "user_rating", "members"]]

In [77]:
search_anime('hi no tori')

,name,genre,type,episodes,anime_rating,user_rating,members
0,hi no tori,"Adventure, Drama, Historical, Sci-Fi, Supernat...",TV,13,7.29,7.439024,4247
1,hi no tori 2772 ai no cosmozone,"Adventure, Drama, Fantasy, Sci-Fi",Movie,1,6.94,7.120000,1717
2,hi no tori hagoromohen,Drama,Movie,1,6.42,5.000000,139
3,hi no tori hououhen,"Drama, Fantasy, Historical",Movie,1,7.13,7.466667,1323
4,hi no tori uchuuhen,"Drama, Fantasy, Sci-Fi",OVA,1,7.19,7.428571,1158
5,hi no tori yamatohen,"Drama, Historical",OVA,1,7.04,7.516129,1069


In [47]:
# def search_anime_by_genres(*genres):
#     """
#     입력한 모든 장르를 포함하는 애니를 검색하는 함수
#     """
#     filtered_anime = df.copy()  # df에서 복사하여 사용
    
#     # 모든 장르를 포함한 애니만 남기기
#     for genre in genres:
#         filtered_anime = filtered_anime[filtered_anime["genre"].str.contains(genre, case=False, na=False)]

#     if filtered_anime.empty:
#         return f"입력한 장르 {genres}를 모두 포함하는 애니를 찾을 수 없습니다."

#     # "anime_rating" 중복 제거 & "user_rating" 평균 처리
#     filtered_anime = filtered_anime.groupby("name", as_index=False).agg({
#         "genre": "first",  # 첫 번째 값 사용
#         "type": "first",  # 첫 번째 값 사용
#         "episodes": "first",  # 첫 번째 값 사용
#         "anime_rating": "first",  # 중복 제거 (첫 번째 값 선택)
#         "user_rating": "mean",  # 중복된 경우 평균 계산
#         "members": "first",  # 첫 번째 값 사용
#     })

#     return filtered_anime[["name", "genre", "type", "episodes", "anime_rating", "user_rating", "members"]]



In [48]:
# search_anime('one punch man')

In [49]:
def search_anime_by_genres(*genres):
    """
    입력한 모든 장르를 포함하는 애니를 검색하는 함수
    """
    filtered_anime = df.copy()  # df에서 복사하여 사용
    
    # 모든 장르를 포함한 애니만 남기기
    for genre in genres:
        filtered_anime = filtered_anime[filtered_anime["genre"].str.contains(genre, case=False, na=False)]

    if filtered_anime.empty:
        return f"입력한 장르 {genres}를 모두 포함하는 애니를 찾을 수 없습니다."

    # "anime_rating" 중복 제거 & "user_rating" 평균 처리
    filtered_anime = filtered_anime.groupby("name", as_index=False).agg({
        "genre": "first",  # 첫 번째 값 사용
        "type": "first",  # 첫 번째 값 사용
        "episodes": "first",  # 첫 번째 값 사용
        "anime_rating": "first",  # 중복 제거 (첫 번째 값 선택)
        "user_rating": "mean",  # 중복된 경우 평균 계산
        "members": "first",  # 첫 번째 값 사용
    })

    return filtered_anime[["name", "genre", "type", "episodes", "anime_rating", "user_rating", "members"]]



In [50]:
search_anime_by_genres("Action", "Comedy", "Parody", "School")

,name,genre,type,episodes,anime_rating,user_rating,members
0,arcade gamer fubuki extra,"Action, Adventure, Comedy, Ecchi, Game, Parody...",Special,1,4.96,5.366667,1039
1,delpower x bakuhatsu miracle genki,"Action, Comedy, Mecha, Parody, School, Sci-Fi",OVA,1,5.68,5.933333,644
2,honoo no tenkousei,"Action, Comedy, Martial Arts, Parody, School, ...",OVA,2,6.77,7.058824,2688
3,ryuuseiki gakusaver,"Action, Comedy, Mecha, Parody, School, Sci-Fi,...",OVA,6,6.67,6.600000,547


---

# Clustering and Collaborative Recommender
##### 클러스터링 및 협업 필터링 추천 시스템
###### 사용자 또는 아이템 간의 유사성을 바탕으로 추천하는 방식

In [51]:
# encoding
le = LabelEncoder()
df['t_genre'] = le.fit_transform(df['genre'])
df['t_type'] = le.fit_transform(df['type'])

In [52]:
df.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'anime_rating',
       'members', 'user_id', 'user_rating', 'first_genre', 't_genre',
       't_type'],
      dtype='object')

In [53]:
len(df)

6337145

In [54]:
# LabelEncoder()는 문자형 데이터를 숫자로 변환하는 scikit-learn의 전처리 클래스입니다.
# 머신러닝 모델은 문자열 데이터를 직접 처리할 수 없기 때문에, 범주형 데이터(카테고리형 데이터)를 숫자로 변환할 때 사용됩니다.
# df['genre'] 열에 있는 각각의 장르 문자열을 고유한 숫자로 변환합니다.
# fit_transform()은 다음 두 가지를 수행합니다.
# fit(): genre 열에 등장하는 고유한 값(카테고리)을 기억합니다.
# transform(): 각 고유한 값(장르)을 정수 값으로 변환합니다.
# 예를 들어, df['genre']에 다음과 같은 값이 있다면:
# ['Action', 'Comedy', 'Drama', 'Action', 'Drama']
# [0, 1, 2, 0, 2] , 숫자는 내부적으로 자동 지정되며, LabelEncoder의 classes_ 속성에서 확인 가능

# 정리
# LabelEncoder()를 사용해 문자 데이터를 숫자로 변환한다.
# fit_transform()은 고유한 값들을 숫자로 매핑하여 새로운 숫자형 컬럼을 생성한다.
# t_genre는 genre의 숫자 변환 결과, t_type은 type의 숫자 변환 결과를 저장한다.
# 이후 클러스터링(Clustering) 및 협업 필터링(Collaborative Filtering)에 활용할 수 있도록 데이터를 정규화한 것이다.

In [55]:
selected_features = ['anime_id', 't_genre', 't_type', 'anime_rating','user_rating'] #클러스터링을 수행할 때 사용할 특징(feature) 컬럼을 선택

# k-means model
n_clusters = 6 # 애니 데이터를 6개의 그룹(클러스터)로 나누도록 설정.
kmeans = KMeans(n_clusters=n_clusters, random_state=42) # 랜덤 시드 고정
df['cluster'] = kmeans.fit_predict(df[selected_features]) #fit_predict()를 사용해 selected_features를 기반으로 K-Means 학습을 진행 후 각 애니메이션 데이터가 속하는 클러스터 ID를 df['cluster']에 저장.

In [56]:
print(df.columns)

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'anime_rating',
       'members', 'user_id', 'user_rating', 'first_genre', 't_genre', 't_type',
       'cluster'],
      dtype='object')


In [57]:
df['cluster'].value_counts()

cluster
0    2396221
5    1165286
3    1097750
1     685069
4     571018
2     421801
Name: count, dtype: int64

In [58]:
from collections import Counter

labels = kmeans.labels_

#count of cluster items
cluster_counts = Counter(labels)

for cluster_id, count in cluster_counts.items():
    print(f'{cluster_id}. 클러스터: {count} elemants')

2. 클러스터: 421801 elemants
5. 클러스터: 1165286 elemants
3. 클러스터: 1097750 elemants
0. 클러스터: 2396221 elemants
1. 클러스터: 685069 elemants
4. 클러스터: 571018 elemants


In [59]:
len(df)

6337145

In [60]:
import random # 랜덤숫자 설정.

# 랜덤한 사용자를 선택하고, 해당 사용자가 속한 클러스터를 찾는 역할을 함.
const_member_index = random.randint(1, len(df)) # df 의 길이 중 랜덤하게 하나 선택택
const_cluster_no = df.cluster[const_member_index] # 위에서 선택한 const_member_index에 해당하는 데이터의 클러스터 번호를 가져옴.
const_cluster_no # 해당 사용자가 속한 클러스터 번호(const_cluster_no)를 저장

4

In [61]:
user_no = df.user_id[const_member_index] #랜덤하게 뽑은 행(const_member_index)에서 해당 user_id를 가져와 user_no에 저장

In [62]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

start_time = time.time()

# 랜덤하게 선택된 사용자(const_member_index)가 속한 클러스터(const_cluster_no)에 속한 데이터만 사용.
df_pivot = df[df.cluster == const_cluster_no].pivot_table(index='name', columns='user_id', values = 'user_rating').fillna(0)

# collaborative filltering metod(KNN모델 학습)
df_matrix = csr_matrix(df_pivot.values) #df_pivot.values를 csr_matrix(희소 행렬)로 변환하여 메모리 최적화.
model_knn = NearestNeighbors(metric='cosine', algorithm='brute') #KNN 모델(NearestNeighbors)을 생성하여 유사도 측정 방식으로 cosine similarity 사용.
model_knn.fit(df_matrix) #KNN 모델을 학습.

# random anime title and finding recommendation
query_no = np.random.choice(df_pivot.shape[0]) # df_pivot에서 임의의 애니메이션을 선택(query_no).
print(f'{query_no}번째 애니메이션인 {df_pivot.index[query_no]}을(를) 본 시청자를 위한 비슷한 애니를 추천하기위해 해당목록을 생성중입니다...')
anime_const = df_pivot.index[query_no]

# KNN을 이용하여 가장 유사한 애니메이션 찾기
distances, indices = model_knn.kneighbors(df_pivot.iloc[query_no, :].values.reshape(1, -1), n_neighbors=10)
# 선택된 애니메이션(query_no)과 가장 유사한 애니메이션 10개(n_neighbors=10)를 찾음.
# distances: 유사도 거리값 (작을수록 유사함).
# indices: 추천된 애니메이션들의 인덱스.

no = []
name = []
distance = []
rating = []
genre = []

#create recommandation
# for i in range(0, len(distances.flatten())):
#     if i == 0:
#         print(f'{df_pivot.index[query_no]}애니 시청자를 위한 추천 애니메이션은 다음과 같습니다... : \n')
#     else:
#         no.append(i)
#         name.append(df_pivot.index[indices.flatten()[i]])
#         distance.append(distances.flatten()[i])
#         rating.append(*df[df['name']==df_pivot.index[indices.flatten()[i]]]['user_rating'].values)
#         genre.append(*df[df['name']==df_pivot.index[indices.flatten()[i]]]['genre'].values)
#         #가장 유사한 애니메이션 리스트를 이름, 평점, 장르, 유사도 순으로 저장.
#         #anime 데이터프레임에서 애니메이션 이름을 기반으로 평점과 장르 정보를 추가.
#create recommandation
for i in range(0, len(distances.flatten())):
    if i == 0:
        print(f'{df_pivot.index[query_no]} 애니 시청자를 위한 비슷한 애니메이션은 다음과 같습니다... : \n')
    else:
        no.append(i)
        name.append(df_pivot.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])

        # user_rating 평균값을 가져오도록 변경
        rating.append(df[df['name'] == df_pivot.index[indices.flatten()[i]]]['user_rating'].mean())

        # genre는 하나의 값만 가져오도록 변경
        genre.append(df[df['name'] == df_pivot.index[indices.flatten()[i]]]['genre'].values[0])

# 추천 결과를 DataFrame으로 변환
dic = {'NO.': no, 'Anime Name': name, 'Rating': rating, 'Genre': genre, 'Similarity': distance[::-1]}
recommendation = pd.DataFrame(data=dic)
recommendation.set_index('NO.', inplace=True)


    #추천 결과를 DataFrame으로 변환
dic = {'NO.': no, 'Anime Name': name, 'Rating' : rating, 'Genre' : genre, 'Similarity' : distance[::-1]}
recommendation = pd.DataFrame(data=dic)
recommendation.set_index('NO.', inplace=True)

end_time = time.time()
elapsed_time = end_time - start_time
print('process time:', elapsed_time, 'sec.')

recommendation.head(10)

514번째 애니메이션인 megumi to taiyou iii kajuu gummi tweet fantasy  timeline world을(를) 본 시청자를 위한 비슷한 애니를 추천하기위해 해당목록을 생성중입니다...
megumi to taiyou iii kajuu gummi tweet fantasy  timeline world 애니 시청자를 위한 비슷한 애니메이션은 다음과 같습니다... : 

process time: 5.629084348678589 sec.


,Anime Name,Rating,Genre,Similarity
NO.,,,,
1,megumi to taiyou kajuu gummi tweet love story,4.5,"Romance, Slice of Life",0.558274
2,megumi to taiyou ii kajuu gummi tweet mystery ...,5.0,Mystery,0.536477
3,fujiko fujio a no mumako,5.0,"Horror, Supernatural",0.441256
4,yamiyo no jidaigeki,4.0,"Historical, Horror",0.429146
5,yamiyo no jidaigeki ova,4.0,"Historical, Horror",0.219131
6,sakura capusule,6.5,"Kids, Slice of Life",0.219131
7,cook no polka,3.0,"Kids, Music",0.219131
8,mogura no adventure,5.0,"Adventure, Kids",0.006116
9,osaru no tairyou,4.0,Slice of Life,0.000000


# Clustring and Content-Based Recommender
### 클러스터링 및 콘텐츠 기반 추천 시스템
###### 애미메이션 시청자가 선호하는 콘텐츠의 특징을 분석하여 유사한 아이템을 추천하는 방식

In [63]:
# create vectorizer
tfv = TfidfVectorizer(analyzer='word') #장르 정보를 TF-IDF 벡터로 변환하여 유사도 행렬을 계산. 단어 단위로 TF-IDF 가중치를 계산.

# get clusters
rec_data = df[df.cluster == const_cluster_no].copy() #K-Means 클러스터 n에 속한 데이터만 가져옴.
#rec_data.drop_duplicates(subset = 'name', keep = 'first', inplace = True)
rec_data = rec_data.groupby('name', as_index=False).agg({# name이 중복된 경우 user_rating과 anime_rating을 평균으로 계산
    'anime_rating': 'mean',  # 애니메이션 전체 평점 평균
    'user_rating': 'mean',   # 사용자 평점 평균
    'genre': 'first'         # 장르는 첫 번째 값 유지
})
rec_data.reset_index(drop = True, inplace = True) #중복 제거 후 인덱스를 초기화하여 데이터프레임을 정리.

# evaluate to genre
#genres = rec_data['name'].str.split(', | , | ,').astype(str)
genres = rec_data['genre'].astype(str) #장르 정보를 문자열로 변환

# create tf-idf matrix
tfv_matrix = tfv.fit_transform(genres) # 장르 간 코사인 유사도 계산

# calculate similarity matrix
cos_sim = cosine_similarity(tfv_matrix, tfv_matrix) #장르 정보를 TF-IDF 벡터로 변환하여 유사도 행렬을 계산.

# drop duplicates
rec_indices = pd.Series(rec_data.index, index = rec_data['name']).drop_duplicates()
#애니메이션 이름(name)을 DataFrame 인덱스로 변환하여 빠르게 검색할 수 있도록 설정.

# # recommendation function 구현 
# def give_recommendation(title, cos_sim=cos_sim):
#     idx = rec_indices[title] # 사용자가 입력한 애니의 인덱스 찾기
#     cos_scores = list(enumerate(cos_sim[idx])) # 입력된 애니와 다른 애니들의 유사도 값 가져오기
#     cos_scores = sorted(cos_scores, key=lambda x: x[1], reverse=True) # 유사도가 높은 순으로 정렬
#     cos_scores = cos_scores[1:11] # 자기 자신(0번째) 제외하고 TOP 10 추천
#     anime_indices = [i[0] for i in cos_scores] # 추천 애니 인덱스 저장

#     # visualization (추천결과를 df에서 가져오기)
#     sim_scores = [i[1] for i in cos_scores]
#     rec_dic = {
#         "No": range(1, 11),
#         "Anime Name": anime["name"].iloc[anime_indices].values,
#         "Rating": anime["rating"].iloc[anime_indices].values,
#         "Genre": anime["genre"].iloc[anime_indices].values,
#         "Similarity Score": sim_scores,
#         # "No": range(1, 11),
#         # "Anime Name": df["name"].iloc[anime_indices].values,  # 추천된 애니의 이름 가져오기.
#         # "User Rating": df["user_rating"].iloc[anime_indices].values,  # 사용자 평점 반영.
#         # "Anime Rating": df["anime_rating"].iloc[anime_indices].values,  # 전체 애니 평점 반영.
#         # "Genre": df["genre"].iloc[anime_indices].values,  # 애니 장르 정보 가져오기.
#         # "Similarity Score": sim_scores,
#     }

#     dataframe = pd.DataFrame(data=rec_dic)
#     dataframe.set_index("No", inplace=True)

#     print(f"'{title}'시청자를 위한 애니 추천 목록... :\n")

#     return dataframe

# recommendation function 구현 (df에서 user_rating 추가)
def give_recommendation(title, cos_sim=cos_sim):
    idx = rec_indices[title]  # 사용자가 입력한 애니의 인덱스 찾기
    cos_scores = list(enumerate(cos_sim[idx]))  # 유사도 값 가져오기
    cos_scores = sorted(cos_scores, key=lambda x: x[1], reverse=True)  # 유사도 순 정렬
    cos_scores = cos_scores[1:11]  # 자기 자신 제외하고 TOP 10 추천
    anime_indices = [i[0] for i in cos_scores]  # 추천 애니 인덱스 저장

    # 🔹 user_rating을 df에서 가져와 평균을 구함
    user_ratings = [
        df[df["name"] == anime["name"].iloc[i]]["user_rating"].mean()
        for i in anime_indices
    ]

    # 추천 결과를 anime에서 가져오기 (하지만 user_rating은 df에서 가져옴)
    sim_scores = [i[1] for i in cos_scores]
    rec_dic = {
        "No": range(1, 11),
        "Anime Name": anime["name"].iloc[anime_indices].values,  # 애니 이름은 anime에서 가져옴
        "User Rating": user_ratings,  # user_rating은 df에서 가져와 평균 적용
        "Anime Rating": anime["rating"].iloc[anime_indices].values,  # 전체 애니 평점 (anime에서 가져옴)
        "Genre": anime["genre"].iloc[anime_indices].values,  # 장르는 anime에서 가져옴
        "Similarity Score": sim_scores,  # 유사도 점수
    }

    # DataFrame 생성
    dataframe = pd.DataFrame(data=rec_dic)
    dataframe.set_index("No", inplace=True)

    print(f"'{title}' 시청자를 위한 애니 추천 목록... :\n")

    return dataframe


In [64]:


start_time = time.time()
clustering_and_content = give_recommendation(anime_const)

end_time = time.time()
elapsed_time = end_time - start_time
print("process time: ", elapsed_time, " sec.")

clustering_and_content

'megumi to taiyou iii kajuu gummi tweet fantasy  timeline world' 시청자를 위한 애니 추천 목록... :

process time:  2.5845260620117188  sec.


,Anime Name,User Rating,Anime Rating,Genre,Similarity Score
No,,,,,
1,mushishi zoku shou odoro no michi,8.556773,8.54,"Adventure, Fantasy, Historical, Mystery, Seine...",1.000000
2,hachimitsu to clover ii,8.492495,8.37,"Drama, Josei, Romance",1.000000
3,junjou romantica 2,8.442641,8.24,"Comedy, Drama, Romance, Shounen Ai",1.000000
4,tokyo ghoul,8.136777,8.07,"Action, Drama, Horror, Mystery, Psychological,...",1.000000
5,date a live encore ova,7.994569,8.01,"Romance, Slice of Life",1.000000
6,hoshi wo ou kodomo,7.926407,7.81,"Adventure, Fantasy, Romance",1.000000
7,jojo no kimyou na bouken phantom blood,7.715686,7.77,"Action, Adventure, Horror, Shounen, Vampire",1.000000
8,major s4,8.406584,8.35,"Comedy, Drama, Shounen, Sports",0.808449
9,baccano specials,8.413591,8.29,"Action, Comedy, Historical, Mystery, Seinen, S...",0.808449


# SVD Recommender

1. SVD (Singular Value Decomposition, 특이값 분해)란?
- SVD는 행렬을 세 개의 행렬의 곱으로 분해하는 기법으로, 추천 시스템에서 널리 사용되는 기법 중 하나야. 기본적으로 SVD는 대규모 행렬의    차원을 줄이면서 중요한 정보만 보존하는 역할을 함.

- 추천 시스템에서의 역할
    사용자-아이템 (예: 유저-애니메이션) 평점 데이터를 행렬 형태로 변환
    희소한 데이터(빈칸이 많은 데이터)에서 패턴을 찾아 예측
    기존 평점 데이터를 기반으로 사용자가 보지 않은 애니메이션의 평점을 예측
    SVD는 넷플릭스 추천 시스템에서도 사용되었을 정도로 강력한 기법

In [65]:
user_no = df.user_id[const_member_index] #결과적으로 user_no에는 df에서 const_member_index에 해당하는 user_id가 저장됨.

In [66]:
user_no

4199

In [67]:
df.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'anime_rating',
       'members', 'user_id', 'user_rating', 'first_genre', 't_genre', 't_type',
       'cluster'],
      dtype='object')

In [68]:
from surprise import SVD # 추천 시스템 구축
from surprise import Dataset, reader
from surprise.model_selection import train_test_split

start_time = time.time()

#create a reader
reader = Reader(rating_scale=(1, 10)) #Reader는 Surprise에서 데이터를 읽는 방식 지정, rating_scale=(1, 10): 평점이 1점에서 10점 사이임을 지정

# get clusters
df_svd = df.copy() #df를 복사하여 df_svd 생성 (원본 데이터 보호), svd 학습을 위해 원본데이터를 그대로 사용

# create data
data = Dataset.load_from_df(df_svd[['user_id', 'name', 'anime_rating']], reader) #Surprise가 사용할 수 있도록 df_svd를 변환, Surprise가 이해할 수 있는 형식으로 변환

# split data
train_set, test_set = train_test_split(data, test_size=.25) #75학습, #25테스트 

# *train SVD model* 사용자-애니메이션 평점 관계를 학습한 모델
model = SVD() 
model.fit(train_set)

predictions = model.test(test_set) # SVD 모델이 실제 평점과 비교하면서 얼마나 정확한지 평가 

end_time = time.time()
elapsed_time = end_time - start_time
print('process time:', elapsed_time, 'sec.')


process time: 91.69427871704102 sec.


In [69]:
# SVD 모델 성능 평가
# performance metrics
accuracy.rmse(predictions) # accuracy.rmse(predictions)는 **RMSE (Root Mean Squared Error, 평균 제곱근 오차)**를 계산.
# RMSE는 예측한 평점과 실제 평점 간의 차이(오차)를 측정하는 지표
# 값이 낮을수록 모델의 예측이 정확하다는 의미.

# 추천 시스템 구현 (추천 알고리즘)
def get_top_n(user_id, n=10): # 사용자가 아직 보지 않은 애니메이션을 찾고, 예측 평점이 높은 순으로 추천하는 방식. (추천받을 사용자 ID, 추천할 애니메이션 개수 n)
    user_animes = df[df['user_id'] == user_id]['name'] # 사용자가 본 에니메이션 목록 불러오기기

    user_unrated_animes = df[~df['name'].isin(user_animes)]['name'] # 사용자가 보지 않은 에니메이션
    user_unrated_animes = list(set(user_unrated_animes))

    predictions = []
    for anime_id in user_unrated_animes:
        predictions.append((anime_id, model.predict(user_id, anime_id).est)) #SVD 모델이 사용자가 해당 애니메이션을 봤을 경우 예측 평점(est)을 계산함. 결과를 리스트 predictiction에 저장.
    predictions.sort(key=lambda x: x[1], reverse=True) #평점 높은 순으로 정렬렬
    top_n = predictions[:n] # 상위 n개의 애니메이션 선택
    top_n = [i[0] for i in top_n] # 제목만 남김.
    return top_n

# create recommendations
recommended_animes = get_top_n(user_no)
recommended_animes

RMSE: 0.0815


['kimi no na wa',
 'gintama°',
 'haikyuu karasuno koukou vs shiratorizawa gakuen koukou',
 'ginga eiyuu densetsu',
 'gintama enchousen',
 'gintama movie kanketsuhen  yorozuya yo eien nare',
 'hunter x hunter 2011',
 'gintama',
 'clannad after story',
 'koe no katachi']

In [70]:
# 장르와 평점 저장할 리스트 
genre_lists, rating_lists = [],[]
seen_names = set()

# 추천된 애니메이션의 장르와 평점 가져오기
for name in recommended_animes:
    if name not in seen_names:
        matched_rows = df[df['name'] == name] 
        genres = list(matched_rows['genre'])
        ratings = list(matched_rows['user_rating'])
        genre_lists.append(genres)
        rating_lists.append(ratings) 
        seen_names.add(name)

# 장르 리스트 변환
type_list = []


for i, genres in enumerate(genre_lists):
    type_list.append(genres[0])

# 평점 리스트 변환
rating_list = []

for i, ratings in enumerate(rating_lists):
    rating_list.append(ratings[0])

# 추천 애니메이션 데이터 생성    
recom_data = {
    'Anime Name':recommended_animes,
    'Rating':rating_list,
    'Genre':type_list}

In [71]:
df_rec = pd.DataFrame(recom_data)
df_rec.head(10)

,Anime Name,Rating,Genre
0,kimi no na wa,5,"Drama, Romance, School, Supernatural"
1,gintama°,10,"Action, Comedy, Historical, Parody, Samurai, S..."
2,haikyuu karasuno koukou vs shiratorizawa gakue...,9,"Comedy, Drama, School, Shounen, Sports"
3,ginga eiyuu densetsu,10,"Drama, Military, Sci-Fi, Space"
4,gintama enchousen,8,"Action, Comedy, Historical, Parody, Samurai, S..."
5,gintama movie kanketsuhen yorozuya yo eien nare,10,"Action, Comedy, Historical, Parody, Samurai, S..."
6,hunter x hunter 2011,9,"Action, Adventure, Shounen, Super Power"
7,gintama,9,"Action, Comedy, Historical, Parody, Samurai, S..."
8,clannad after story,9,"Drama, Fantasy, Romance, Slice of Life, Supern..."
9,koe no katachi,8,"Drama, School, Shounen"


# Hybrid (K-Means + SVD) Recommender
##### 비슷한 취향을 가진 사용자 그룹을 만든 후, 그 그룹 내에서 SVD 모델을 사용하여 추천을 수행
###### 특정 클러스터(유사한 사용자 그룹) 내에서 학습.
###### 해당 클러스터에서 랜덤으로 선택한 사용자(user_no).
###### 같은 클러스터 내 사용자들의 미평가 애니메이션에 대한 예상 평점.

In [72]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

start_time = time.time()
 
# create a reader
reader = Reader(rating_scale=(1, 10))

# 특정 클러스터 데이터만 선택 (K-Means 적용)
df_hybrid = df[df.cluster==const_cluster_no].copy()

# create data (surprise 형식으로)
data = Dataset.load_from_df(df_hybrid[['user_id', 'name', 'user_rating']], reader)

# split data
trainset, testset = train_test_split(data, test_size=.25,)

# train SVD model
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
model.fit(trainset)

# create predictions
predictions = model.test(testset)

end_time = time.time()
elapsed_time = end_time - start_time
print("process time: ", elapsed_time, " sec.")

process time:  6.794448375701904  sec.


In [73]:
# performance metrics
accuracy.rmse(predictions)
# create recommendations
recommended_animes = get_top_n(user_no)
recommended_animes

RMSE: 1.1336


['jojo no kimyou na bouken stardust crusaders 2nd season',
 'kuroko no basket 3rd season',
 'shirobako',
 'jojo no kimyou na bouken stardust crusaders',
 'nanatsu no taizai',
 'noragami',
 'hajime no ippo rising',
 'uchuu senkan yamato 2199 hoshimeguru hakobune',
 'akatsuki no yona',
 'ping pong the animation']

In [74]:
genre_lists, rating_lists = [],[]
seen_names = set()
for name in recommended_animes:
    if name not in seen_names:
        matched_rows = df[df['name'] == name] 
        genres = list(matched_rows['genre'])
        ratings = list(matched_rows['user_rating'])
        genre_lists.append(genres)
        rating_lists.append(ratings)
        seen_names.add(name)

        
type_list = []

for i, genres in enumerate(genre_lists):
    type_list.append(genres[0])
    
rating_list = []

for i, ratings in enumerate(rating_lists):
    rating_list.append(ratings[0])
    
recom_data = {
    'Anime Name':recommended_animes,
    'Rating':rating_list,
    'Genre':type_list}

In [75]:
df_rec = pd.DataFrame(recom_data)
df_rec.head(10)

,Anime Name,Rating,Genre
0,jojo no kimyou na bouken stardust crusaders 2n...,8,"Action, Adventure, Drama, Shounen, Supernatural"
1,kuroko no basket 3rd season,10,"Comedy, School, Shounen, Sports"
2,shirobako,9,"Comedy, Drama"
3,jojo no kimyou na bouken stardust crusaders,8,"Action, Adventure, Drama, Shounen, Supernatural"
4,nanatsu no taizai,8,"Action, Adventure, Ecchi, Fantasy, Shounen, Su..."
5,noragami,8,"Action, Adventure, Shounen, Supernatural"
6,hajime no ippo rising,8,"Comedy, Drama, Shounen, Sports"
7,uchuu senkan yamato 2199 hoshimeguru hakobune,8,"Action, Drama, Military, Sci-Fi, Space"
8,akatsuki no yona,8,"Action, Adventure, Comedy, Fantasy, Romance, S..."
9,ping pong the animation,10,"Psychological, Seinen, Sports"


---

In [ ]:
import pickle
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

df_pivot = df.pivot_table(index='name', columns='user_id', values='user_rating').fillna(0)

df_matrix = csr_matrix(df_pivot.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
model_knn.fit(df_matrix)

with open("knn_model.pkl", "wb") as f:
    pickle.dump(model_knn, f)

with open("df_pivot.pkl", "wb") as f:
    pickle.dump(df_pivot, f)

✅ KNN 모델 학습 완료 및 저장됨: knn_model.pkl, df_pivot.pkl
